In [2]:
%reset -f


pip install pandas beancount matplotlib seaborn


In [3]:
from beancount.loader import load_file
import pandas as pd

entries, _, _ = load_file("./tab-data/ledger/dynamic.beancount")

txns = []
for entry in entries:
    if entry.__class__.__name__ == "Transaction":
        txn_id = id(entry)  # new unique ID for this transaction
        for posting in entry.postings:
            txns.append({
                "transaction_id": txn_id,
                "date": entry.date,
                "narration": entry.narration,
                "meta": entry.meta,
                "account": posting.account,
                "units": posting.units.number,
                "currency": posting.units.currency,
            })

df = pd.DataFrame(txns)


In [4]:
df

,transaction_id,date,narration,meta,account,units,currency
0,124767734339664,1970-01-01,Initial balance transfer for Liabilities:Bar:M...,{'filename': '/home/mateo/projects/bar-graphs/...,Liabilities:Bar:Members:Bloemist,5.30,EUR
1,124767734339664,1970-01-01,Initial balance transfer for Liabilities:Bar:M...,{'filename': '/home/mateo/projects/bar-graphs/...,Assets:InitialBalances,-5.30,EUR
2,124767734340336,1970-01-01,Initial balance transfer for Liabilities:Bar:M...,{'filename': '/home/mateo/projects/bar-graphs/...,Liabilities:Bar:Members:Koen,-0.50,EUR
3,124767734340336,1970-01-01,Initial balance transfer for Liabilities:Bar:M...,{'filename': '/home/mateo/projects/bar-graphs/...,Assets:InitialBalances,0.50,EUR
4,124767734341008,1970-01-01,Initial balance transfer for Liabilities:Bar:M...,{'filename': '/home/mateo/projects/bar-graphs/...,Liabilities:Bar:Members:Bart,-11.80,EUR
...,...,...,...,...,...,...,...
104612,124767574668816,2026-01-22,Mateo bought 2 items for €2.50,{'filename': '/home/mateo/projects/bar-graphs/...,Liabilities:Bar:Members:Mateo,1,SODA
104613,124767574668816,2026-01-22,Mateo bought 2 items for €2.50,{'filename': '/home/mateo/projects/bar-graphs/...,Assets:Inventory:Bar,-1,SNACK
104614,124767574668816,2026-01-22,Mateo bought 2 items for €2.50,{'filename': '/home/mateo/projects/bar-graphs/...,Liabilities:Bar:Members:Mateo,1,SNACK
104615,124767574668816,2026-01-22,Mateo bought 2 items for €2.50,{'filename': '/home/mateo/projects/bar-graphs/...,Liabilities:Bar:Members:Mateo,2.50,EUR


# Examining columns

In [5]:
for column in df.columns:
    try:
        unique_count = df[column].nunique()
    except TypeError:
        unique_count = df[column].apply(lambda x: str(x)).nunique()
    print(f"{column}: {unique_count} unique values")


transaction_id: 25841 unique values
date: 2173 unique values
narration: 9474 unique values
meta: 25841 unique values
account: 92 unique values
units: 865 unique values
currency: 81 unique values


# Get rid of meta column

In [6]:
# Show first row
first_row = df.iloc[1000]
# print(first_row)

# Show shortened meta (first 100 chars)
meta_short = str(first_row['meta'])
print(meta_short)


{'filename': '/home/mateo/projects/bar-graphs/tab-data/ledger/imported-from-spacebar.beancount', 'lineno': 1713, '__tolerances__': {'EUR': Decimal('0.005')}}


From the meta we ONLY want the type of the purchase

In [7]:
df['type'] = df['meta'].apply(lambda m: m.get('type') if isinstance(m, dict) else None)
df = df.drop(columns=['meta'])
# Show results
df['type'].unique()



<StringArray>
[nan, 'purchase', 'deposit', 'transfer']
Length: 4, dtype: str

In [8]:
df

,transaction_id,date,narration,account,units,currency,type
0,124767734339664,1970-01-01,Initial balance transfer for Liabilities:Bar:M...,Liabilities:Bar:Members:Bloemist,5.30,EUR,NaN
1,124767734339664,1970-01-01,Initial balance transfer for Liabilities:Bar:M...,Assets:InitialBalances,-5.30,EUR,NaN
2,124767734340336,1970-01-01,Initial balance transfer for Liabilities:Bar:M...,Liabilities:Bar:Members:Koen,-0.50,EUR,NaN
3,124767734340336,1970-01-01,Initial balance transfer for Liabilities:Bar:M...,Assets:InitialBalances,0.50,EUR,NaN
4,124767734341008,1970-01-01,Initial balance transfer for Liabilities:Bar:M...,Liabilities:Bar:Members:Bart,-11.80,EUR,NaN
...,...,...,...,...,...,...,...
104612,124767574668816,2026-01-22,Mateo bought 2 items for €2.50,Liabilities:Bar:Members:Mateo,1,SODA,purchase
104613,124767574668816,2026-01-22,Mateo bought 2 items for €2.50,Assets:Inventory:Bar,-1,SNACK,purchase
104614,124767574668816,2026-01-22,Mateo bought 2 items for €2.50,Liabilities:Bar:Members:Mateo,1,SNACK,purchase
104615,124767574668816,2026-01-22,Mateo bought 2 items for €2.50,Liabilities:Bar:Members:Mateo,2.50,EUR,purchase


# Examining transactions

Each transaction is a collection of postings.

In [9]:
postings_per_transaction = df.groupby("transaction_id").size().reset_index(name="num_postings")

postings_per_transaction = postings_per_transaction.sort_values(
    by='num_postings',
    ascending=False
).reset_index(drop=True)

top_id = postings_per_transaction.iloc[0]["transaction_id"]

postings_per_transaction


,transaction_id,num_postings
0,124767596323088,17
1,124767577492272,12
2,124767680516880,12
3,124767597147104,12
4,124767653314736,12
...,...,...
25836,124767734346384,2
25837,124767734345712,2
25838,124767734345040,2
25839,124767734344368,2


Let's look at all postings in this transaction

In [10]:
df[df["transaction_id"] == top_id]


,transaction_id,date,narration,account,units,currency,type
88846,124767596323088,2023-08-03,Els bought 30 items for €118.00,Assets:Inventory:Bar,-1,COCKTAIL,purchase
88847,124767596323088,2023-08-03,Els bought 30 items for €118.00,Liabilities:Bar:Members:Els,1,COCKTAIL,purchase
88848,124767596323088,2023-08-03,Els bought 30 items for €118.00,Assets:Inventory:Bar,-3,GERMAN,purchase
88849,124767596323088,2023-08-03,Els bought 30 items for €118.00,Liabilities:Bar:Members:Els,3,GERMAN,purchase
88850,124767596323088,2023-08-03,Els bought 30 items for €118.00,Assets:Inventory:Bar,-6,DUVEL,purchase
88851,124767596323088,2023-08-03,Els bought 30 items for €118.00,Liabilities:Bar:Members:Els,6,DUVEL,purchase
88852,124767596323088,2023-08-03,Els bought 30 items for €118.00,Assets:Inventory:Bar,-8,CHIP,purchase
88853,124767596323088,2023-08-03,Els bought 30 items for €118.00,Liabilities:Bar:Members:Els,8,CHIP,purchase
88854,124767596323088,2023-08-03,Els bought 30 items for €118.00,Assets:Inventory:Bar,-8,WATER,purchase
88855,124767596323088,2023-08-03,Els bought 30 items for €118.00,Liabilities:Bar:Members:Els,8,WATER,purchase


# Get products

In [11]:
import yaml

# Load YAML and convert to DataFrame
with open("./tab-data/static/products.yml", "r") as f:
    price_data = yaml.safe_load(f)
df_products = pd.DataFrame(price_data)

# Expand the 'payback' dictionary into its own columns
if 'payback' in df_products.columns:
    payback_df = df_products['payback'].apply(lambda x: x if isinstance(x, dict) else {}).apply(pd.Series)
    payback_df.columns = [f'payback_{col}' for col in payback_df.columns]
    df_products = pd.concat([df_products.drop(columns=['payback']), payback_df], axis=1)

df_products

,name,price,event_price,currency,category,visible,payback_account,payback_amount
0,Soda,1.5,2.50,SODA,drink,True,NaN,NaN
1,Cecemel,0.7,1.00,CHOCO_MILK,drink,False,NaN,NaN
2,Fritz,2.5,3.50,GERMAN,drink,True,NaN,NaN
3,Club Mate,3.0,4.00,CM,drink,True,NaN,NaN
4,Tea,0.5,0.75,TEA,drink,False,NaN,NaN
...,...,...,...,...,...,...,...,...
63,Pizza,2.5,4.00,PIZZA,food,False,Liabilities:Bar:Members:Yvan,1.5
64,SPRITE,2.0,3.00,SPRITE,DRINK,False,NaN,NaN
65,Cha-Cha,0.5,0.75,CHACHA,food,False,NaN,NaN
66,Leffe,2.0,3.00,LEFFE,alcohol,False,NaN,NaN


See all products where a payback is present. This means a member should be paid back every time an items is purchased of this kind.

In [12]:
df_products[df_products['payback_account'].notna()]

,name,price,event_price,currency,category,visible,payback_account,payback_amount
11,Wristband,1.0,1.5,WRISTBAND,swag,True,Liabilities:Bar:Members:Marquis,0.50
14,Passport,2.5,3.6,PASSPORT,swag,True,Liabilities:Bar:Members:Bart,1.20
15,Postcard,2.5,3.5,POSTCARD,NaN,NaN,Liabilities:Bar:Members:NewlineJar,1.00
59,ROM,1.0,1.5,ROM,food,False,Liabilities:Bar:Members:Thequux,0.50
62,Ice Cream,1.0,1.5,ICE_CREAM,food,False,Liabilities:Bar:Members:Yvan,0.63
63,Pizza,2.5,4.0,PIZZA,food,False,Liabilities:Bar:Members:Yvan,1.50


# Get recent values

In [13]:
df["date"] = pd.to_datetime(df["date"])  # Convert first
cutoff = pd.Timestamp.now() - pd.DateOffset(years=3)
recent_df = df[df["date"] >= cutoff]    # Then filter

In [14]:
recent_df

,transaction_id,date,narration,account,units,currency,type
80472,124767669283056,2023-01-24,qwaxys deposited €60.00,Liabilities:Bar:Members:Bart,-60.00,EUR,deposit
80473,124767669283056,2023-01-24,qwaxys deposited €60.00,Assets:Cash:Bar,60.00,EUR,deposit
80474,124767669283616,2023-01-24,qwaxys gave --CASH-- a gift of €60.00,Liabilities:Bar:Members:Bart,60.00,EUR,transfer
80475,124767669283616,2023-01-24,qwaxys gave --CASH-- a gift of €60.00,Assets:Cash:Bar,-60.00,EUR,transfer
80476,124767669284400,2023-01-24,Fearless bought 1 items for €1.50,Assets:Inventory:Bar,-1,SODA,purchase
...,...,...,...,...,...,...,...
104612,124767574668816,2026-01-22,Mateo bought 2 items for €2.50,Liabilities:Bar:Members:Mateo,1,SODA,purchase
104613,124767574668816,2026-01-22,Mateo bought 2 items for €2.50,Assets:Inventory:Bar,-1,SNACK,purchase
104614,124767574668816,2026-01-22,Mateo bought 2 items for €2.50,Liabilities:Bar:Members:Mateo,1,SNACK,purchase
104615,124767574668816,2026-01-22,Mateo bought 2 items for €2.50,Liabilities:Bar:Members:Mateo,2.50,EUR,purchase
